# Stage 1 : Spectral peaks

In [2]:
import matplotlib.pyplot as plt
import librosa.display

import numpy as np
import pandas as pd
import librosa

In [3]:
def find_spectral_peaks(filename):
    print("fsp")
    y, sr = librosa.load(filename)
    D = np.abs(librosa.stft(y))
    tst_array = D
    max_filter_size = [91, 65]
    for x_init in range(0, tst_array.shape[0], max_filter_size[0]):
        for y_init in range(0, tst_array.shape[1], max_filter_size[1]):
            subarr = tst_array[x_init:(x_init + max_filter_size[0]), y_init:(y_init + max_filter_size[1])]
            mx = np.amax(subarr)
            tst_array[x_init:(x_init + max_filter_size[0]), y_init:(y_init + max_filter_size[1])] = np.where(subarr != mx, 0, subarr)
    # print("MAX FILTER APPLIED:")
    min_filter_size = [3, 3]
    for x_init in range(0, tst_array.shape[0], min_filter_size[0]):
        for y_init in range(0, tst_array.shape[1], min_filter_size[1]):
            subarr = tst_array[x_init:(x_init + min_filter_size[0]), y_init:(y_init + min_filter_size[1])]
            mn = np.amin(subarr)
            tst_array[x_init:(x_init + min_filter_size[0]), y_init:(y_init + min_filter_size[1])] = np.where(subarr != mn, subarr, 0)
    # print("MIN FILTER APPLIED:")
    return np.count_nonzero(tst_array), D

In [4]:
def get_peak_coordinates_from_matrix(matrix):
    print("gpfm")
    coordinates = []
    for i in range(matrix.shape[0]):
        for col in range(matrix.shape[1]):
            if matrix[i][col] != 0:
                coordinates.append((i, col))
   # print(coordinates)
    return coordinates

In [29]:
#Creating Quads
def create_quads(spectral_peaks_array):
        #STEP - 1: Using the x,y values of the peaks, sort it based on time
        print("cq")
        quads = []
        spectral_peaks_array = sorted(spectral_peaks_array)
        spectral_peaks_array = np.array(spectral_peaks_array)
        print(spectral_peaks_array)
        #STEP -2: define a value k and then define r with k being the mid point
        for root in spectral_peaks_array:
            Ax,Ay = 600, 1000
            k = 10
            r = 8
            n = 0
            start = int(Ax + (k - r/2))
            end = int(Ax + k + r/2)
            np.where(spectral_peaks_array > (start, Ay), spectral_peaks_array, 0)
            print("considered points")
            print(spectral_peaks_array)
            break
#             range_peaks = []
#             for i in range(start, end):
#                 range_peaks.append(sorted_peaks[i])
#                 n += 1
#             #STEP -3: chose a peak value which should denote B 
#             if(n <= 3):
#                 # discard and move to the next root node
#                 continue
#             range_peaks = sorted(range_peaks)
#             B = range_peaks[-1]
#             #STEP -4: Using the above steps a bounding box has been created, now within the bounding box apply the 4 conditions
#             for j in range_peaks:
#                 if(A[1] < B[1]):
#                     for z in range(j+1,range_peaks):
#                         C = j
#                         D = z
#                         if(A[0] < C[0] and C[0]<= D[0] and D[0] <= B[0]):
#                             if(A[1] < C[1] and C[1]< D[1] and D[1] < B[1]):
#                                 B_x_off = B[0] - Ax
#                                 B_y_off = B[1] - Ay
#                                 C_x_off = C[0] - Ax
#                                 C_y_off = C[1] - Ay
#                                 D_x_off = D[0] - Ax
#                                 D_y_off = D[1] - Ay
#                 #STEP -5: Append the four values [A,B,C,D] in the Quad array

#                                 quads.append([(A[0],A[1]),(B_x_off,B_y_off),(C_x_off,C_y_off),(D_x_off,D_y_off)])

        #STEP -4: Repeat for all peaks and create quads array
        return quads

# Stage 2 : Storing hashes

In [6]:
import sqlite3
from rtree import index

In [7]:
conn = sqlite3.connect('quad.db')
print("Opened database successfully")

Opened database successfully


In [8]:
conn.execute('''CREATE TABLE UNNORMALIZED_QUADS
         (ID INT PRIMARY KEY     NOT NULL AUTOINCREMENT,
         AX  INT                 NOT NULL,
         AY  INT                 NOT NULL,
         BX  INT                 NOT NULL,
         BY  INT                 NOT NULL,
         CX  INT                 NOT NULL,
         CY  INT                 NOT NULL,
         DX  INT                 NOT NULL,
         DY  INT                 NOT NULL);''')


OperationalError: table UNNORMALIZED_QUADS already exists

In [9]:
conn.execute('''CREATE TABLE FIDINDEX
         (FID INT PRIMARY KEY    NOT NULL,
         FILENAME TEXT           NOT NULL,
         QUADIDX_START INT       NOT NULL,
         QUADIDX_END   INT       NOT NULL);''')

OperationalError: table FIDINDEX already exists

In [10]:
# TODO : Complete this function
def quads_sanity_check(quads):
    return 1

In [15]:
def db_insert_quads_record(file, quads):
    if quads_sanity_check(quads) == 0:
        return
    cur = conn.cursor()
    cur.execute("SELECT ID FROM UNNORMALIZED_QUADS ORDER BY ID DESC LIMIT 1")
    row = cur.fetchall()
    if not row:
        start_id = 0
    start_id = 200
    for a in quads:
        cur.execute("INSERT INTO UNNORMALIZED_QUADS VALUES ({}, {}, {}, {}, {}, {}, {}, {}, {})".format(
                    (start_id+1), a[0][0], a[0][1], a[1][0], a[1][1], a[2][0], a[2][1], a[3][0], a[3][1]))
        start_id += 1
    return row

# System simulations

In [12]:
import os
import time
path = '/home/supraraj/Projects/AudioFinger/audio_files/'

audio_input_files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.mp3' in file:
            audio_input_files.append(os.path.join(r, file))

In [30]:
for file in audio_input_files:
    print("")
    print("File: {}".format(file))
    start = time.time()
    peak_count, peaks = find_spectral_peaks(file)
    peaks_coordinates = get_peak_coordinates_from_matrix(peaks)
    quads_created = create_quads(peaks_coordinates)
    # expected to get a list of tuples of the form [[(x,y), (bx_off,by_off), (cx_off,cy_off), (dx_off,dy_off)], ...] 
    #quads = [[(3,4), (10, 40), (2, 5), (3, 15)],
        #         [(6,7), (8, 23), (1, 1), (2, 2)]]
    
    #row = db_insert_quads_record(file, quads)
    #print(row)
    #end = time.time()
    break


File: /home/supraraj/Projects/AudioFinger/audio_files/Nico & Vinz - Am I Wrong [Official Music Video].mp3
fsp
gpfm
cq
[[    3  4096]
 [    3  8317]
 [    3  8322]
 ...
 [ 1007 10219]
 [ 1007 11660]
 [ 1011   904]]
considered points
[[    3  4096]
 [    3  8317]
 [    3  8322]
 ...
 [ 1007 10219]
 [ 1007 11660]
 [ 1011   904]]


In [18]:
a = np.arange(100).reshape(20,5)
a

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24],
       [25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34],
       [35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44],
       [45, 46, 47, 48, 49],
       [50, 51, 52, 53, 54],
       [55, 56, 57, 58, 59],
       [60, 61, 62, 63, 64],
       [65, 66, 67, 68, 69],
       [70, 71, 72, 73, 74],
       [75, 76, 77, 78, 79],
       [80, 81, 82, 83, 84],
       [85, 86, 87, 88, 89],
       [90, 91, 92, 93, 94],
       [95, 96, 97, 98, 99]])

In [35]:
(90, 100) > (100, 90)

False